In [1]:
import requests
from bs4 import BeautifulSoup

all_links = []

# WHO
base_url = "https://www.who.int"
extract_url = "https://www.who.int/news-room/fact-sheets"
response = requests.get(extract_url)
soup = BeautifulSoup(response.text, "html.parser")
for a in soup.find_all("a", href=True):
    href = a["href"]
    if "/news-room/fact-sheets/detail/" in href:   # WHO-specific pattern
        full_url = base_url + href if href.startswith("/") else href
        all_links.append(full_url)

# NHS
extract_url2 = "https://www.nhs.uk/conditions/"
response = requests.get(extract_url2)
soup = BeautifulSoup(response.text, "html.parser")
for a in soup.find_all("a", href=True):
    href = a["href"]
    if href.startswith("/conditions/"):   # NHS-specific pattern
        full_url = "https://www.nhs.uk" + href
        all_links.append(full_url)

# CDC
extract_url3 = "https://www.cdc.gov/health-topics.html"
response = requests.get(extract_url3)
soup = BeautifulSoup(response.text, "html.parser")
for a in soup.find_all("a", href=True):
    href = a["href"]
    if "/diseases-conditions/" in href or "/topics/" in href:  # CDC topics pattern
        full_url = "https://www.cdc.gov" + href if href.startswith("/") else href
        all_links.append(full_url)

# MedlinePlus
extract_url4 = "https://medlineplus.gov/all_healthtopics.html"
response = requests.get(extract_url4)
soup = BeautifulSoup(response.text, "html.parser")
for a in soup.find_all("a", href=True):
    href = a["href"]
    if href.startswith("/"):   # MedlinePlus topics are internal links
        full_url = "https://medlineplus.gov" + href
        all_links.append(full_url)

# Mayo Clinic
extract_url5 = "https://www.mayoclinic.org/diseases-conditions"
response = requests.get(extract_url5)
soup = BeautifulSoup(response.text, "html.parser")
for a in soup.find_all("a", href=True):
    href = a["href"]
    if "/diseases-conditions/" in href:
        full_url = "https://www.mayoclinic.org" + href if href.startswith("/") else href
        all_links.append(full_url)

print(len(all_links))
print(all_links[:10])  # preview first 10 links


1087
['https://www.who.int/news-room/fact-sheets/detail/abortion', 'https://www.who.int/news-room/fact-sheets/detail/abuse-of-older-people', 'https://www.who.int/news-room/fact-sheets/detail/adolescents-health-risks-and-solutions', 'https://www.who.int/news-room/fact-sheets/detail/adolescent-pregnancy', 'https://www.who.int/news-room/fact-sheets/detail/ageing-and-health', 'https://www.who.int/news-room/fact-sheets/detail/alcohol', 'https://www.who.int/news-room/fact-sheets/detail/ambient-(outdoor)-air-quality-and-health', 'https://www.who.int/news-room/fact-sheets/detail/anaemia', 'https://www.who.int/news-room/fact-sheets/detail/animal-bites', 'https://www.who.int/news-room/fact-sheets/detail/antimicrobial-resistance']


In [3]:
from langchain_community.document_loaders import UnstructuredURLLoader

In [5]:
loader=UnstructuredURLLoader(urls=all_links)

In [7]:
my_docs=loader.load()

In [17]:
my_docs[0].page_content[500:700]  # preview characters 500 to 700 of the first document

'\n\nY\n\nZ\n\nRegions\n\nAfrica\n\nAmericas\n\nEurope\n\nEastern Mediterranean\n\nSouth-East Asia\n\nWestern Pacific\n\nWHO in countries\n\nData by country\n\nCountry presence\n\nCountry strengthening\n\nCountry cooperation stra'

In [18]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=2000,
    chunk_overlap=500
)
my_chunks = text_splitter.split_documents(my_docs)

In [19]:
my_chunks[0].page_content[:500]  # preview first 500 characters of the first chunk

'Skip to main content\n\nWhen autocomplete results are available use up and down arrows to review and enter to select.\n\nHome\n\nAll topics\n\nA\n\nB\n\nC\n\nD\n\nE\n\nF\n\nG\n\nH\n\nI\n\nJ\n\nK\n\nL\n\nM\n\nN\n\nO\n\nP\n\nQ\n\nR\n\nS\n\nT\n\nU\n\nV\n\nW\n\nX\n\nY\n\nZ\n\nResources\n\nFact sheets\n\nFacts in pictures\n\nMultimedia\n\nPodcasts\n\nPublications\n\nQuestions and answers\n\nTools and toolkits\n\nPopular\n\nDengue\n\nEndometriosis\n\nExcessive heat\n\nHerpes\n\nMental disorders\n\nMpox\n\nAll countries\n\nA\n\nB\n\nC\n\nD\n\nE\n\nF\n\nG\n\nH\n\nI\n\nJ\n\nK\n\nL\n\nM\n\nN\n\nO\n\nP\n\nQ\n\nR\n\nS\n\nT\n\nU\n\nV\n\nW\n\nX'

In [21]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


d:\Medical\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [23]:
vector_store.add_documents(documents=my_chunks)

['b4f8093f-1c0f-4064-9931-36be348e0cd6',
 '25019d5c-384d-4bda-8b3c-b70a916a183c',
 '00b5c5a0-0aff-4012-bef0-059a6e17d80e',
 'e03fba94-f89c-461c-b98c-226dc80a23b4',
 '95505d78-1526-48a3-878a-55633d566885',
 'af8514cf-b01a-4f58-86d7-5ed3822aa27c',
 '08c78c0a-70df-4dd7-aa98-8547c73cedfc',
 '54279df8-d76c-4ac4-9289-6606c4376280',
 'a2a17ec2-5e8d-4c14-9e51-9ccfac0c9679',
 '145729d7-2736-4bf4-82e0-12f6840e4c4c',
 '49861e83-f935-4724-9936-18866467ea90',
 '2bd57691-9f81-4af4-af4a-2c75d9251da7',
 '27c6b171-322a-481b-94e5-45a3cd69ce27',
 'a1adb92e-5a8c-460d-be09-4295db19b04c',
 'c1da0d14-d8ac-44fb-b786-c1074f7eaa60',
 'ad56b274-8a3a-4841-9876-e5b5eece6a3f',
 '202b2f3b-1369-4dfc-a60f-f1e821607c7a',
 'b2869f3a-b33e-4100-8405-b85bd5099b63',
 'd51f7182-4c37-407b-bd78-93512c8b6979',
 'a323ed41-e5f3-4262-ac54-b605c429c3e7',
 '0e5fc27c-0484-41d6-88b5-5612e7b84736',
 'bb1ba120-9c53-4b30-918d-7533bdd2133d',
 '4c59ad1e-4462-41d2-bd98-ca06d61ea6d3',
 '3979f162-2658-4b3b-b38a-2ae6299ae856',
 '89c1d280-2475-

In [27]:
vector_store.save_local("faiss_index")